In [16]:
import matplotlib.pyplot as plt
import numpy as np
from itertools import chain
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

from qiskit import BasicAer, QuantumCircuit
from qiskit.circuit.library import ZZFeatureMap
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import QSVM
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name
from qiskit.circuit.library import QFT

seed = 10599
aqua_globals.random_seed = seed
DIM = 28
NUM_QUBITS = 10

In [17]:
dataset = np.load('../data/images.npy')[:500]
dataset_labels = np.load('../data/labels.npy')[:500]
print('No of entries in dataset {}'.format(len(dataset)))

No of entries in dataset 500


In [18]:
print(dataset.shape)
dataset = PCA(n_components=2).fit_transform(dataset.reshape(-1,DIM*DIM))
print(dataset.shape)
X_train, X_test, y_train, y_test = train_test_split(dataset, dataset_labels, test_size=0.33, random_state=seed)
print('shape of X_train {}'.format(X_train.shape))
print('shape of y_train {}'.format(y_train.shape))
print('shape of X_test {}'.format(X_test.shape))
print('shape of y_test {}'.format(y_test.shape))
print('Size of train set {}'.format(len(X_train)))
print('Size of test set {}'.format(len(X_test)))

(500, 28, 28)
(500, 2)
shape of X_train (335, 2)
shape of y_train (335,)
shape of X_test (165, 2)
shape of y_test (165,)
Size of train set 335
Size of test set 165


In [19]:
def encode_img(image, register_num):
    ''' encoding of image using QPIE method '''
    img = list(chain(*image))
    pix_val = img

    # normalize
    pix_norm = np.linalg.norm(pix_val)
    pix_val = np.array(pix_val)
    arr_norm = pix_val/pix_norm
    arr_norm = arr_norm.tolist()

    # Encode onto the quantum register
    qc = QuantumCircuit(register_num)
    # test = arr_norm.append(np.zeros(2**10-arr_norm.shape))
    test = arr_norm + np.zeros(2**register_num-DIM**2).tolist()
    qc.initialize(test, qc.qubits)
    return qc


def encode(image):
    ''' final wrapper function (for submission) '''
    return apply_qft(encode_img(255*255*image, register_num=NUM_QUBITS), register_num=NUM_QUBITS)
    # return encode_img(255*255*image, register_num=NUM_QUBITS)

# apply qft


def apply_qft(circuit, register_num):
    circuit.append(
        QFT(register_num, do_swaps=False).to_gate(), circuit.qubits)
    return circuit

In [20]:
training_input = {'0': [], '1': []}
test_input = {'0': [], '1': []}

for i in range(len(y_train)):
    if y_train[i] == True:
        training_input['1'].append(X_train[i])
        # print(training_input['1'][-1].shape)
        # assert False
    else:
        training_input['0'].append(X_train[i])
# print(training_input)
# print(X_train)
# print(y_train)
for i in range(len(y_test)):
    if y_test[i] == True:
        test_input['1'].append(X_test[i])
    else:
        test_input['0'].append(X_test[i])

In [21]:
feature_map = ZZFeatureMap(feature_dimension=2, reps=1, entanglement='linear')
# enc_image = encode_img(X_train[0])
qsvm = QSVM(feature_map, training_input, test_input)

backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1, seed_simulator=seed, seed_transpiler=seed)

result = qsvm.run(quantum_instance)

print(f'Testing success ratio: {result["testing_accuracy"]}')
print()
# print('Prediction from datapoints set:')
# print(f'  ground truth: {map_label_to_class_name(datapoints[1], qsvm.label_to_class)}')
# print(f'  prediction:   {result["predicted_classes"]}')
# predicted_labels = result["predicted_labels"]
# print(f'  success rate: {100*np.count_nonzero(predicted_labels == datapoints[1])/len(predicted_labels)}%')

Testing success ratio: 1.0



/home/varshaneya/iquhack/ionqenv/lib/python3.6/site-packages/qiskit/aqua/algorithms/classifiers/qsvm/_qsvm_binary.py:109: ComplexWarning: Casting complex values to real discards the imaginary part
  predicted_labels = binarized_predictions.astype(int)


In [22]:
import pickle
with open('quantum_classifier_500samples.pkl','wb') as f:
    pickle.dump(qsvm, f)

TypeError: Required argument 'file' (pos 2) not found